In [ ]:
import sys
!{sys.executable} -m pip install rdflib owlrl

# utilities
import pandas as pd

# libraries to handle triples and graphs
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import RDFS, RDF, OWL

# libraries to handle reasoning
from owlrl import DeductiveClosure, OWLRL_Semantics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.9 MB/s eta 0:00:00


# Loading Graphs

In [ ]:
# utility variable : list of namespaces that we will need for querying
namespaces = {'rdf': RDF, 'rdfs' : RDFS, "": 'http://example.org/', 'owl': OWL}

In [ ]:
# initialise an empty graph
myKnowledgeGraph = Graph()

# parse a knowledge graph from a local file
myKnowledgeGraph = Graph().parse("./sample_data/kitchen-exercise.ttl")

FileNotFoundError: [Errno 2] No such file or directory: '/sample_data/kitchen-exercise.ttl'

Let's see how many triples the graph contains

In [ ]:
print("Graph has %s statements." % len(myKnowledgeGraph))

Graph has 211 statements.


Let's try adding and removing triples to see what changes in the graph.

In [ ]:
# Adding triples
myKnowledgeGraph.add((URIRef("http://example.org/bed2"),RDF.type , URIRef("http://example.org/Bed")))

# print the length of the graph
print("My Kitchen graph has now %d triples." % len(myKnowledgeGraph))

#removing triples
myKnowledgeGraph.remove((URIRef("http://example.org/bed2"),RDF.type , URIRef("http://example.org/Bed")))

# print again the length of the graph
print("My Kitchen graph has now %d triples." % len(myKnowledgeGraph))

My Kitchen graph has now 212 triples.
My Kitchen graph has now 211 triples.


# Querying with RDFlib (without reasoning)

We can now query the graph we created. We start by asking which subjects are there in the graph.

In [ ]:
# this is equivalent to saying : for i in myKnowledgeGraph.subjects() : print i
list(i for i in myKnowledgeGraph.subjects() if type(i) != BNode)

[rdflib.term.URIRef('http://example.org/table2'),
 rdflib.term.URIRef('https://example.org/Box'),
 rdflib.term.URIRef('http://example.org/Wardrobe'),
 rdflib.term.URIRef('http://example.org/sink1'),
 rdflib.term.URIRef('http://example.org/room2'),
 rdflib.term.URIRef('http://example.org/wardrobe1'),
 rdflib.term.URIRef('http://example.org/room3'),
 rdflib.term.URIRef('http://example.org/sink1'),
 rdflib.term.URIRef('http://example.org/fridge1'),
 rdflib.term.URIRef('http://example.org/box1'),
 rdflib.term.URIRef('http://example.org/Object'),
 rdflib.term.URIRef('http://example.org/room1'),
 rdflib.term.URIRef('http://example.org/bookshelf1'),
 rdflib.term.URIRef('http://example.org/Bed'),
 rdflib.term.URIRef('http://example.org/bookshelf2'),
 rdflib.term.URIRef('http://example.org/bed1'),
 rdflib.term.URIRef('http://example.org/hasStatus'),
 rdflib.term.URIRef('http://example.org/Kitchen'),
 rdflib.term.URIRef('http://example.org/table4'),
 rdflib.term.URIRef('http://example.org/stove1

In [ ]:
sorted(list(kitchen_class for kitchen_class in myKnowledgeGraph.subjects(RDF.type, OWL.Class) if type(kitchen_class) != BNode))

[rdflib.term.URIRef('http://example.org/Bed'),
 rdflib.term.URIRef('http://example.org/BedRoom'),
 rdflib.term.URIRef('http://example.org/Chair'),
 rdflib.term.URIRef('http://example.org/Door'),
 rdflib.term.URIRef('http://example.org/Fridge'),
 rdflib.term.URIRef('http://example.org/Kitchen'),
 rdflib.term.URIRef('http://example.org/KitchenObject'),
 rdflib.term.URIRef('http://example.org/LivingRoom'),
 rdflib.term.URIRef('http://example.org/Location'),
 rdflib.term.URIRef('http://example.org/Object'),
 rdflib.term.URIRef('http://example.org/Room'),
 rdflib.term.URIRef('http://example.org/Shelf'),
 rdflib.term.URIRef('http://example.org/Sink'),
 rdflib.term.URIRef('http://example.org/Sofa'),
 rdflib.term.URIRef('http://example.org/Stove'),
 rdflib.term.URIRef('http://example.org/Table'),
 rdflib.term.URIRef('http://example.org/Wardrobe'),
 rdflib.term.URIRef('https://example.org/Box')]

Let's ask whether 'door2' is closed.



In [ ]:
openDoor2 = myKnowledgeGraph.value(URIRef('http://example.org/door3'), URIRef("http://example.org/hasStatus"))

if openDoor2 == "closed":
  print(True)
else : print(False)

False


We can update the status of a door and ask again if there is any door open.

In [ ]:
# NB: you need to remove the old triple from the graph, and then add a new one. Else they will both be stored in the KG!
myKnowledgeGraph.remove((URIRef("http://example.org/door2"),URIRef('http://example.org/hasStatus') , Literal("open")))
myKnowledgeGraph.add((URIRef("http://example.org/door2"),URIRef('http://example.org/hasStatus') , Literal("closed")))


# ask which doors are closed
for s,p,o in myKnowledgeGraph.triples((None, URIRef("http://example.org/hasStatus"), None )):
  if o == Literal("closed") : print(s)

http://example.org/door2


# Reasoning and querying

We can now start a reasoner and generate triples we did not know before.

In [ ]:
# initialise new graphs : asserted (triples stated in the ttl file), inferred (triples generated by the reasoner)
asserted = Graph()
inferred = Graph()

asserted = asserted.parse("./sample_data/kitchen-exercise.ttl")
DeductiveClosure(OWLRL_Semantics).expand(myKnowledgeGraph) # this function will run an OWL RL reasoner and expand the asserted KG with inferred triples
# NB : the .expand() function will add the inferred triples to the original asserted graph. Thankfully, we saved a copy of the original asserted KG into the variable 'asserted'!

# we can now check which are the inferred triples by substracting the expanded and original graph.
inferred = myKnowledgeGraph - asserted

Let's now check how many triples these graphs have.

In [ ]:
print("asserted {}, inferred {}, total {}".format(len(asserted), len(inferred),len(myKnowledgeGraph)))


asserted 211, inferred 589, total 737


Based on the restriction

```
:Kitchen owl:equivalentClass [ rdf:type owl:Restriction ;
                               owl:onProperty :contains ;
                               owl:someValuesFrom ( :Fridge :Sink :Stove )
                              ]
```
we know that the class `Kitchen` should contain at least an element from the class `Fridge`, `Sink` or `Stove`.

From the asserted triples, we know that `:room2` contains `:fridge1`, `:stove1` and `:sink1` .

If the reasoner worked, we can ask the inferred graph whether `:room2` is a kitchen.

In [ ]:
EX = Namespace("http://example.org/")

# NB : EX['room2'] is now equivalent to URIRef("http://example.org/room2")
for s,p,o in inferred.triples((EX['room2'],RDF.type,None)):
      print(s,p,o)

http://example.org/room2 http://www.w3.org/1999/02/22-rdf-syntax-ns#type n625f8bf0e68249f69c578e9e6e87c91db10
http://example.org/room2 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://example.org/Kitchen
http://example.org/room2 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://example.org/Location
http://example.org/room2 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Thing


Now compare with what the asserted graph knows about `:room2`.   

In [ ]:
# NB : EX['room2'] is now equivalent to URIRef("http://example.org/room2")
for s,p,o in asserted.triples((EX['room2'],RDF.type,None)):
      print(s,p,o)

http://example.org/room2 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#NamedIndividual
http://example.org/room2 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://example.org/Room


The reasoner also reasons about object property contraints.

For instance, we know that the property `:contains` is inverse of `:hasLocation`.

In [ ]:
# select objects from room4
for s,p,o in asserted.triples( (EX['room4'],EX['contains'],None)  ):
  print (o)

# Try replacing the triple pattern above with the triple (and remember to print the subject instead of the object)

Nothing is known about `:room4` in the asserted graph. Hence the result is empty.

Go check yourself in the `.ttl` file!



In [ ]:
# compare with what is known about room4 in the inferred graph
for s,p,o in inferred.triples( (EX['room4'],EX['contains'],None)  ):
  print (o)

# Now we have the right inference!

http://example.org/bookshelf3
http://example.org/box3
http://example.org/box2
http://example.org/table1
http://example.org/table2


# Exercises

**Exercise 1**

Extend the taxonomy of Kitchen classes, with e.g. movable and non-movable classes

- Create a new file .ttl and parse it in a new graph.

- Query the new graph and check if your taxonomy was taken into account.




In [ ]:
# your code here

**Exercise 2**

Build a restriction for the living room or bedroom, similar to the one we have for the kitchen
(i.e. a bedroom is a room that contains at least one bed, a livingroom has at least a sofa and a bookshelf)

- Create a new file .ttl and parse it in a new graph.

- Run the reasoner and check if your restriction was taken into account.

- _harder variant_ : try using ```owl:minCardinality``` instead of ```owl:someValuesFrom``` ! Tip : you may need to create a new object property.



In [ ]:
# your code here

**Exercise 3**

Update the status of a door (from e.g. open to close) or the coordinates of an object.

- Use the functions
```
graph.remove((UriRef(#subject),UriRef(#predicate ),UriRef( #object )))
graph.add((UriRef(#subject),UriRef(#predicate ),UriRef( #object )) )
```
to remove the old triple and add a new one.



- Save the graph using the function
```graph.serialise()```

- check with your favourite text editor if your KG has been updated!

In [ ]:
# your code here